In [16]:
%load_ext autoreload
%autoreload 2
import os
import copy
import pickle
from functools import lru_cache
import pandas as pd
import copy
import time
import sys
from statistics import mean

import numpy as np
import scipy
from tqdm import tqdm

from gensim.models import Word2Vec
from gensim import utils
from gensim.corpora import Dictionary
from gensim.models.lsimodel import LsiModel
from sklearn.preprocessing import MinMaxScaler

import sys
sys.path.append('../src')
from models import calculate_association_metric_for_target_word, word_set_to_mtx, get_1storder_association_metric


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
we_model_name = "sg_dim300_min100_win5"
we_vector_size = 300
we_model_dir = '../data/external/wiki-english/wiki-english-20171001/%s' % we_model_name

In [4]:
we_model = Word2Vec.load(we_model_dir+'/model.gensim')

print ('loading done!')

loading done!


In [5]:
#loading E_ctx_vec
with open(we_model_dir+'/E_ctx_vec.pkl', 'rb') as fr:
    E_ctx_vec = pickle.load(fr)
with open(we_model_dir+'/E_wrd_vec.pkl', 'rb') as fr:
    E_wrd_vec = pickle.load(fr)

# Calculating *d*

In [16]:
# WEAT 1
X_terms = ['aster', 'clover', 'hyacinth', 'marigold', 'poppy', 'azalea', 
           'crocus', 'iris', 'orchid', 'rose', 'bluebell', 'daffodil','lilac',
           'pansy','tulip','buttercup','daisy','lily','penny','violet','carnation',
           'magnolia','petunia','zinnia']
Y_terms = ['caterpillar','flea','locust','spider','bedbug','centipede','fly',
          'maggot','tarantula','bee','cockroach','gnat','mosquito','termite','beetle',
          'cricket','hornet','moth','wasp','blackfly','dragonfly','horsefly','roach',
          'weevil']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']
# Removed words:
# X: gladiola (not found)
# Y: ant (random)

In [18]:
# WEAT 2
X_terms = ['bagpipe','cello','guitar','lute','trombone','banjo','clarinet','harmonica',
           'mandolin','trumpet','bassoon','drum','harp','oboe','tuba','bell','fiddle',
           'harpsichord','piano','viola','bongo','flute','horn','saxophone']
Y_terms = ['arrow','club','gun','missile','spear','axe','dagger','harpoon','pistol',
          'sword','blade','dynamite','hatchet','rifle','tank','bomb','firearm',
          'knife','shotgun','teargas','cannon','grenade','mace','slingshot','whip']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'divorce','jail','poverty','ugly','cancer','kill','rotten','vomit','agony',
          'prison']

In [34]:
# WEAT 3
X_terms = ['roger','alan','frank','ian','justin',
          'ryan','andrew','fred','jack','matthew','stephen','brad','greg','jed',
          'paul','todd','brandon','hank','jonathan','peter','wilbur','amanda',
          'courtney','heather','melanie','sara','amber','crystal','katie',
          'meredith','shannon','betsy','donna','kristin','nancy','stephanie',
          'bobbie-sue','ellen','lauren','peggy','sue-ellen','colleen','emily',
          'megan','rachel','wendy']
Y_terms = ['alonzo','jamel','lerone','theo','alphonse','jerome','leroy',
           'torrance','darnell','lamar','lionel','rashaun','tyree','deion',
          'lamont','malik','terrence','tyrone','lavon','terryl',
          'wardell','aiesha','lashelle','nichelle','shereen','temeka','ebony',
          'latisha','shaniqua','tameisha','teretha','jasmine','laronya','shanise',
          'tanisha','tia','lakisha','latoya','sharise','tashika','yolanda',
          'lashandra','malika','shavonn','tawanda','yvette']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
          'loyal','pleasure','diamond','gentle','honest','lucky','rainbow',
          'diploma','gift','honor','miracle','sunrise','family','happy','laughter',
          'paradise','vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy','bomb',
          'divorce','jail','poverty','ugly','cancer','evil','kill','rotten','vomit']
print ("Missing x terms: ", [x for x in X_terms if x not in we_model.wv])
print('missing y terms: ', [x for x in Y_terms if x not in we_model.wv])
# Removed terms
# X: adam (random), chip (random), harry (random), josh (random)
# Y: percell (not found), rasaan (not found), everol (not found), marcelius (not found)

Missing x terms:  ['bobbie-sue', 'sue-ellen']
missing y terms:  ['terryl', 'aiesha', 'lashelle', 'temeka', 'tameisha', 'teretha', 'laronya', 'shanise', 'lakisha', 'sharise', 'tashika', 'lashandra', 'shavonn']


In [47]:
# WEAT 4
X_terms = ['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil',
'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith',
           'Sarah']
Y_terms = ['Darnell', 'Hakim', 'Jermaine', 'Kareem', 'Jamal', 'Leroy', 'Rasheed',
'Tremayne', 'Tyrone', 'Aisha', 'Ebony', 'Keisha', 'Kenya', 'Latonya', 'Latoya',
           'Tamika', 'Tanisha']
A_terms = ['caress','freedom','health','love','peace','cheer','friend','heaven',
           'loyal','pleasure','diamond','gentle','honest','lucky','rainbow','diploma',
           'gift','honor','miracle','sunrise','family','happy','laugher','paradise',
           'vacation']
B_terms = ['abuse','crash','filth','murder','sickness','accident','death','grief',
          'poison','stink','assault','disaster','hatred','pollute','tragedy',
          'bomb','divorce','jail','poverty','ugly','cancer','evil','kill',
          'rotten','vomit']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]


In [45]:
# WEAT 5
X_terms = ['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil',
'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith',
           'Sarah']
Y_terms = ['Darnell', 'Hakim', 'Jermaine', 'Kareem', 'Jamal', 'Leroy', 'Rasheed',
'Tremayne', 'Tyrone', 'Aisha', 'Ebony', 'Keisha', 'Kenya', 'Latonya', 'Latoya',
           'Tamika', 'Tanisha']
A_terms = ['joy', 'love', 'peace', 'wonderful', 'pleasure', 'friend', 'laughter', 'happy']
B_terms = ['agony', 'terrible', 'horrible', 'nasty', 'evil', 'war', 'awful', 'failure']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]
# Removed terms: "Lakisha"

In [5]:
# WEAT 6
X_terms = ['John', 'Paul','Mike','Kevin','Steve','Greg','Jeff','Bill']
Y_terms = ['Amy','Joan','Lisa','Sarah','Diana','Kate','Ann','Donna']
A_terms = ['executive','management','professional','corporation',
               'salary','office','business','career']
B_terms = ['home','parents','children','family',
               'cousins','marriage','wedding','relatives']
[X_terms, Y_terms, A_tersm, B_terms] = [tuple([str.lower(term) for term in terms]) for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [8]:
# WEAT 7 using Google News word2vec model
X_terms = ['math','algebra','geometry','calculus',
             'equations','computation','numbers','addition']
Y_terms = ['poetry','art','dance','literature',
             'novel','symphony','drama','sculpture']
A_terms = ['male','man','boy','brother',
              'he','him','his','son']
B_terms = ['female','woman','girl','sister',
               'she','her','hers','daughter']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [33]:
# Weat 8
A_terms = ['brother','father','uncle','grandfather',
          'son','he','his','him']
B_terms = ['sister','mother','aunt','grandmother',
          'daughter','she','hers','her']
X_terms = ['science','technology','physics','chemistry',
          'Einstein','NASA','experiment','astronomy']
Y_terms = ['poetry','art','Shakespeare','dance',
          'literature','novel','symphony','drama']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

In [35]:
# WEAT 9
X_terms = ['sad','hopeless','gloomy','tearful','miserable','depressed']
Y_terms = ['sick','illness','influenza','disease','virus','cancer']
A_terms = ['impermanent','unstable','variable','fleeting','short','brief',
          'occasional']
B_terms = ['stable','always','constant','persistent','chronic','prolonged','forever']

In [7]:
# Weat 10
X_terms = ['Tiffany', 'Michelle', 'Cindy', 'Kristy', 'Brad', 'Eric', 'Joey', 'Billy']
Y_terms = ['Ethel', 'Bernice', 'Gertrude', 'Agnes', 'Cecil', 'Wilbert', 'Mortimer', 'Edgar']
A_terms = ['joy', 'love', 'peace', 'wonderful', 'pleasure', 'friend', 'laughter', 'happy']
B_terms = ['agony', 'terrible', 'horrible', 'nasty', 'evil', 'war', 'awful', 'failure']
[X_terms, Y_terms, A_tersm, B_terms] = [[str.lower(term) for term in terms] for terms in [X_terms, Y_terms, A_terms, B_terms]]

# Generating List of Association Metrics

# Second-Order

In [7]:
def get_association_metrics_for_term_list(wv_obj, target_terms, A_terms, B_terms):
    target_term_mtx = word_set_to_mtx(wv_obj, target_terms)
    A_mtx = word_set_to_mtx(wv_obj, A_terms)
    B_mtx = word_set_to_mtx(wv_obj, B_terms)
    return np.apply_along_axis(lambda x_vec: calculate_association_metric_for_target_word(x_vec, A_mtx, B_mtx), 1, target_term_mtx)

In [9]:
get_association_metrics_for_term_list(we_model, X_terms, A_terms, B_terms)

array([-0.00392048, -0.00072572,  0.01474717,  0.01163117,  0.00974191,
        0.00101295,  0.00605045, -0.00280538])

# First-Order

In [21]:
def get_1storder_association_metrics_for_term_list(wv_obj, target_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec):
    [E_ctx_vec_tup, E_wrd_vec_tup, A_terms, B_terms] = [tuple(E_ctx_vec), tuple(E_wrd_vec), tuple(A_terms), tuple(B_terms)]
    x_associations = np.array([])
    for x in tqdm(target_terms):
        x_association = get_1storder_association_metric(x, A_terms, B_terms, we_model, E_ctx_vec_tup, E_wrd_vec_tup)
        x_associations = np.append(x_associations, x_association)
    return x_associations
get_1storder_association_metrics_for_term_list(we_model, X_terms, A_terms, B_terms, E_ctx_vec, E_wrd_vec)

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 11.24it/s]


array([-0.21382576, -0.0455426 ,  0.05146244,  0.06901419,  0.05146657,
        0.01058954,  0.0239864 , -0.27391827])